In which I attempt to control multiple robots simultaneously!


In [ ]:
# Foundational Sphero API, forked from Simon's Library
import sphero

In [ ]:
# Tracking code
import cv2
# SpheroNav base classes
from tracker.trackingfilter import FilterSpheroBlueCover, FilterSpheroYellowCover, FilterGlow

from tracker.traceable import TraceableObject
from tracker.trackerbase import ColorTracker
from util import Vector2D

In [ ]:
# Support notebook usage of SpheroTeam
import sys
sys.path.append('../')

In [ ]:
import SpheroTeam
# If project is rearchitected, rethink how to make functions accessible
# Look into python module / api design
from SpheroTeam.teamutil import readJsonFile, normalize_angle, angle_between_points
from SpheroTeam.navigation import calibrate_bot_direction, get_bot_position, get_team_offsets

from SpheroTeam.formations import roll_polygon, roll_push
from SpheroTeam.director import bot_go_to_point, team_go_to_points, team_go_to_paths


In [ ]:
# General Python Imports
import time
import os
import logging

#### SpheroTeam Setup


For more details about what each line does, refer to the previous notebook (6)


In [ ]:
# This is a list of the colored "initial names" of the spheros you want to work with in this session
ACTIVEBOTS = ['RPB', 
              'RWR', 
              'ORG',
             'YYP', 
             'RYR',
             'YPR',
             'GRY']

In [ ]:
from SpheroTeam import initialize

In [ ]:
PROJ_ROOT = os.pardir
CONFIG = os.path.join(PROJ_ROOT, "config.json")
config = readJsonFile(CONFIG)
TEAM_COLORS = config['teamColors']

# Initialize to manage sphero objects
manager = sphero.SpheroManager()

# Initialize Sphero manager using options provided in the config file
manager= initialize.load_sphero_roster(manager, CONFIG)


# Connect to robots
# if you omit ACTIVEBOTS, by default it tries to import all 7
# bots = initialize.connect_sphero_team(manager, ACTIVEBOTS)
bots = initialize.connect_sphero_team(manager)

In [ ]:
# Check power, robots shut down after dropping below 7 volts

SpheroTeam.print_team_status(bots)

In [ ]:
# Check robot colors from config file
# RGB, followed by 
TEAM_COLORS = config['teamColors']
TEAM_COLORS

In [ ]:
SpheroTeam.set_team_colors(bots, TEAM_COLORS)


#### Camera Tracker Setup



In [ ]:
tracker = ColorTracker(config['cameraID'])


In [ ]:
imageX, imageY = tracker.image_size
print "Image: {} x {} pixels".format(imageX, imageY)



In [ ]:
# For now, blue is the only color we can reliably track
traceable_color = [0, 0, 255]  # RGB Blue

traceable_blue = TraceableObject("BLUE")
traceable_blue.filter = FilterSpheroBlueCover()

In [ ]:
# make sure everyone's light is off before calibrating.
for bot in bots:
    bot.set_rgb(0,0,0)
    

In [ ]:
# Blue seems to be the most reliable color to track with.
# Be suspicious if any of the offsets are 0!!!
# the offsets of just a few select robots


offsets = get_team_offsets(bots, traceable_blue, traceable_color, tracker)

In [ ]:
# to redo individual offsets, do this in-place modification of the offset
def team_redo_offset(offsets, bots, index, 
                traceable_object, traceable_color, tracker):
    """
        If some of the offsets are bad, redo the offset 
        for bot at index
        i without losing the others
    """
    offsets[index] = calibrate_bot_direction(bots[index], traceable_object, 
                                             traceable_color, tracker)
    
    return offsets

In [ ]:
offsets = team_redo_offset(offsets, bots, 3, traceable_blue, traceable_color, tracker)


In [ ]:
?roll_polygon

In [ ]:
# Roll in a square to show it's working
# can roll in single direction using roll-push.
# roll_polygon(bots, 3, offsets, 0, speed=50, TIMEOUT=1000)


In [ ]:
for i, bot in enumerate(bots):
#     offsets = team_redo_offset(offsets, bots, i, traceable_blue, traceable_color, tracker)
    SpheroTeam.roll_sphero(bot, 60, 0, offsets[i])
    
    time.sleep(1.5)
    
    
    
    
    
    
    

In [ ]:
?SpheroTeam.roll_sphero


In [ ]:
# Debug individual robots
iShift = 3
SpheroTeam.roll_sphero(bots[iShift], 60, 0, offsets[iShift])

In [ ]:
SpheroTeam.set_team_colors(bots, TEAM_COLORS)

In [ ]:
# Square Video

roll_polygon(bots, 4, offsets, 0, speed=120, TIMEOUT=1000)

In [ ]:
# Up down left right
roll_polygon(bots, 1, offsets, 180, speed=60, TIMEOUT=2000)
time.sleep(1.5)

roll_polygon(bots, 1, offsets, 0, speed=60, TIMEOUT=1500)
time.sleep(1.5)

roll_polygon(bots, 1, offsets, 90, speed=60, TIMEOUT=1500)
time.sleep(1.5)


roll_polygon(bots, 1, offsets, 270, speed=60, TIMEOUT=1500)
time.sleep(1.5)

### Finally: Group sending of robots to points




One problem: when robot rolls off the viewing field. Maybe when a robot cannot be seen, it should roll in the opposite of the last angle it rolled in.


In [ ]:
?get_bot_position()

In [ ]:
print imageX
print imageY

In [ ]:
# Every robot, after some pushing activity, will use this path to skirt the perimeter
recoveryPaths = [
    [
        [320, 400],  # 
        [100, 245]
    ],
    
    [
        [320, 100],
        [100, 235]
    ],
    
    [
        [320, 400],  # 
        [100, 250]
    ],
    
    [
        [320, 100],
        [100, 230]
    ],
    
    [
        [320, 400],  # 
        [100, 255]
    ],
    
    [
        
        [320, 100],
        [100, 225]
    ],
    [
        [320, 100],
        [100, 225]
    ]
]

In [ ]:
# Debugging individual robot navigation
# team_go_to_points([bots[2]], [[50, 50]], [offsets[2]], traceable_blue, traceable_color, stopRadius=15, )

In [ ]:
from SpheroTeam.navigation import get_rectangle_targets


In [ ]:
# Send robots to a cluster around 300, 150
targets = [ [300, 150 + i*5] for i in range(len(bots))] # space out the robots along x = 100




# targets = get_rectangle

team_go_to_points(bots, targets, offsets, traceable_blue, traceable_color, imageX, imageY, tracker, stopRadius=15, Kp=.26)


### Now, move them together




In [ ]:
SpheroTeam.set_team_colors(bots, TEAM_COLORS)    


In [ ]:
# Test a pentagon
roll_polygon(bots, 5, offsets, 180, speed=60, TIMEOUT=1200)

In [ ]:
# Bring a lost sphero back if it rolled away (consider making an arena wall in future)
# SpheroTeam.roll_sphero(bots[0], 40, 180, offsets[0])

In [ ]:
?roll_push

In [ ]:
# Roll all team along angle 0 at a speed of 70 for 3 seconds
roll_push(bots, 0, 70, offsets, TIMEOUT=3000)



In [ ]:
# Show how robots would recover from "overshooting" the target by going to the outside of the arena
#, and then returning to a home area.


team_go_to_paths(bots, recoveryPaths, offsets, traceable_blue, traceable_color, imageX, imageY, tracker)




Challenges: 

- Path planning is not the same thing as simply choosing to arrive at a point.
    - Need to come up with algorithm to avoid obstacles, knowing that you may miss goal points by 10-20 centimeters.
    
- Tuning the right KP parameter is tricky
- Coming up with recovery paths is tricky

- How to stop robots from bumping into each other


- When robot runs out of bounds, we can't see it... need to create a "fence" around the arena and that would fix it.

- Tracker can only reliably "see" 1 color at once, so workaround is only using that 1 color for tracking.

Alternate ideas that weren't implemented due to lack of time

- When swarm needs to converge on 1 point: have everyone drive at fixed speed

- drive for however long that distance takes in pixels

- Let everyone drive simultaneously by giving each robot a blind directive, and not using any feedback to tune its behavior (aka a "formation")


- Write a function that given a singular (x,y) point, generates a list of pixels that each robot would aim for, along with an "shape" and relevant parameters for how the robots should be arranged (aka box, line, circle, hexagon, etc)

- Write a function that given a singular target (x,y) point and a robot, define a list of points that the robot should visit before reaching the target points

